In [1]:
## Importing all relevant libraries

import random

import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score


from transformers import pipeline
from transformers import AutoTokenizer, AutoModel, AutoModelForPreTraining
from transformers import BigBirdConfig

import torch

import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [2]:
df_new = pd.read_csv("/project/class/bii_sdad_dspg/uva_2021/binn/DNA_Food_Articles_2017_no_rep.csv", index_col=0) ## Reading in data
#df_save = df.reset_index(inplace=False, drop = True)
df_new = df_new.reset_index(inplace=False, drop = True)
#df_new = df_new.iloc[0:20,]
df_new.head(2)

,index,copyright,subject_codes,art,modification_datetime,body,company_codes_occur_ticker_exchange,company_codes_occur,company_codes_about,company_codes_lineage,...,company_codes,industry_codes,title,publication_datetime,publisher_name,action,document_type,date,year,full_text
0,15,© Copyright 2017 Journal of Engineering via Ve...,",ccat,",NaN,1489442488000,Those respondents with advanced full life cycl...,NaN,",comasc,",",comasc,",NaN,...,",comasc,comasc,comasc,",",imobsoft,i3302,i330202,i3302021,icomp,itech,",CA Technologies; Are You Getting the Most Valu...,1489363200000,"NewsRX, LLC",add,article,2017-03-13 00:00:00.000,2017,CA Technologies; Are You Getting the Most Valu...
1,18,© Copyright 2017 Computer Weekly News via via ...,",ccat,",NaN,1487890959000,The Access Valve for ThingWorx integrates an o...,NaN,NaN,NaN,NaN,...,NaN,",isecpri,iioft,i3302,i330202,i3302021,i3303,ic...","Certified Security Solutions, Inc. CSS Release...",1487808000000,"NewsRX, LLC",add,article,2017-02-23 00:00:00.000,2017,"Certified Security Solutions, Inc. CSS Release..."


In [4]:
df = pd.read_excel("/project/class/bii_sdad_dspg/uva_2021/binn/Data_600_Labeled_Final.xlsx") ## Reading in data
#df_save = df.reset_index(inplace=False, drop = True)
df = df.reset_index(inplace=False, drop = True)
df.head(2)

,Unnamed: 0,an,title,snippet,body,full_text,"Does it mention an innovation (yes,no,unsure)?",What is the company?,"Where is company mentioned (title,body,snippet,all)?",What is the product name?,"Where is the product name mentioned(title,body,snippet,all)","What could indicate that this is a product (for use in our algorithms, e.g. TM,R,C,CAPITAL LETTERS, Quotes,Used with certain key phrases or words etc.)?","Features of the product, if mentioned",assignee,YN_INNOVATION
0,0,PNN0000020180426ee4p00006,Creaform and Metrologic Group Announce Worldwi...,Metrologic Group and Creaform have entered int...,Speed and volumetric accuracy\n\nThe CUBE-R™ e...,Creaform and Metrologic Group Announce Worldwi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diggy,yes
1,1,INVDAI0020150827eb8r00011,"4 Growth Stocks Make Case For ""New Leader"" Status",The IPO Leaders screen can hold up to 16 stock...,"Still, it makes sense to fill your watch list ...","4 Growth Stocks Make Case For ""New Leader"" Sta...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diggy,no


In [5]:
## Balancing target class 

df_yes = df[df.YN_INNOVATION == 'yes']
df_no = df[df.YN_INNOVATION == 'no']

if len(df_yes) == len(df_no):
    df_yes = df_yes.append(df_no)
    df = df_yes
    
elif len(df_yes) > len(df_no):
    df_yes = df_yes.sample(n=len(df_no), random_state = 65432)
    df_yes = df_yes.append(df_no)
    df = df_yes
    
elif len(df_no) > len(df_yes):
    df_no = df_no.sample(n=len(df_yes), random_state = 65432)
    df_yes = df_yes.append(df_no)
    df = df_yes
    
df = df.sample(n=len(df), random_state = 65432).reset_index(inplace=False, drop = True)

    
print("Value counts for target variable:\n",df.YN_INNOVATION.value_counts())

## Encoding target variable

le = preprocessing.LabelEncoder()
df.YN_INNOVATION = le.fit_transform(df.YN_INNOVATION)
print("\n\n\nThe target variable classes are:", le.classes_)

Value counts for target variable:
 no     222
yes    222
Name: YN_INNOVATION, dtype: int64



The target variable classes are: ['no' 'yes']


In [6]:
## NLP Models from Hugging Face, pretrained_weights object is changed according to required model
model_class, tokenizer_class, pretrained_weights = (AutoModel, 
                                                    AutoTokenizer, 
                                                    'distilbert-base-uncased')
                                                 
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

#Tokenize
tokenized = df['full_text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, padding=True, 
                                                          truncation=True,max_length=512)))


#Train Data
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

np.array(padded).shape
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
    
    
    


#Test/Train  
features = last_hidden_states[0][:,0,:].numpy()
labels = df['YN_INNOVATION']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, random_state = 65432)

#Logistic Regression
lr_clf = LogisticRegression(solver="sag")
lr_clf.fit(train_features, train_labels)

#Accuracy
lr_clf.score(test_features, test_labels)



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/far3jh/.conda/envs/DiggyConda/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


0.8918918918918919

In [7]:
def predict_innovation(df_new):
    tokenized_new =  df_new['full_text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, padding=True, truncation=True,max_length=512)))
    max_len = 0
    for i in tokenized_new.values:
        if len(i) > max_len:
            max_len = len(i)

    padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized_new.values])

    np.array(padded).shape
    attention_mask = np.where(padded != 0, 1, 0)
    attention_mask.shape

    input_ids = torch.tensor(padded)  
    attention_mask = torch.tensor(attention_mask)
    
    #print(chunk)

    with torch.no_grad():
        last_hidden_states_new = model(input_ids, attention_mask=attention_mask)

    features_new = last_hidden_states_new[0][:,0,:].numpy()




    predicted_y = lr_clf.predict(features_new)
    pred_prob = lr_clf.predict_proba(features_new)[:, 1]
    index = df_new['an']

    df_final = pd.DataFrame(list(zip(index, predicted_y, pred_prob)), columns = ["an", "pred_y", "pred_prob"])
    return df_final


In [8]:
results = pd.DataFrame(columns = ["an", "pred_y", "pred_prob"])
results.to_csv('/project/class/bii_sdad_dspg/uva_2021/binn/100k_articles_index_innovation_classification_prediction_diggy.csv')
i = 1
for chunk in np.array_split(df_new, 1000):
    print("The current chunk is: ", i)
    var = predict_innovation(chunk)
    var.to_csv('/project/class/bii_sdad_dspg/uva_2021/binn/100k_articles_index_innovation_classification_prediction_diggy.csv', mode='a', header=False)
    i = i + 1

The current chunk is:  1
The current chunk is:  2
The current chunk is:  3
The current chunk is:  4
The current chunk is:  5
The current chunk is:  6
The current chunk is:  7
The current chunk is:  8
The current chunk is:  9
The current chunk is:  10
The current chunk is:  11
The current chunk is:  12
The current chunk is:  13
The current chunk is:  14
The current chunk is:  15
The current chunk is:  16
The current chunk is:  17
The current chunk is:  18
The current chunk is:  19
The current chunk is:  20
The current chunk is:  21
The current chunk is:  22
The current chunk is:  23
The current chunk is:  24
The current chunk is:  25
The current chunk is:  26
The current chunk is:  27
The current chunk is:  28
The current chunk is:  29
The current chunk is:  30
The current chunk is:  31
The current chunk is:  32
The current chunk is:  33
The current chunk is:  34
The current chunk is:  35
The current chunk is:  36
The current chunk is:  37
The current chunk is:  38
The current chunk is: